In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('train (1).csv')

In [3]:
df.sample(5)

,id,keyword,location,text,target
5279,7544,oil%20spill,Corpus Christi,'California: Spring Oil Spill Estimate Grows '...,1
1164,1676,bridge%20collapse,"New Delhi, Delhi",Ashes 2015: AustraliaÛªs collapse at Trent Br...,1
1744,2510,collision,Sacramento,South Sac Florin Rd / Franklin Blvd **Trfc Col...,1
3525,5039,eyewitness,"Bakersfield, California",Wake Up Kern County Eyewitness News Mornings a...,1
7577,10829,wrecked,#NewcastleuponTyne #UK,@widda16 ... He's gone. You can relax. I thoug...,0


In [4]:
df['location'].unique()

array([nan, 'Birmingham', 'Est. September 2012 - Bristol', ...,
       'Vancouver, Canada', 'London ', 'Lincoln'], dtype=object)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [6]:
df.fillna(method='ffill')

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,wrecked,Lincoln,Two giant cranes holding a bridge collapse int...,1
7609,10870,wrecked,Lincoln,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,wrecked,Lincoln,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,wrecked,Lincoln,Police investigating after an e-bike collided ...,1


In [7]:
df['location'].value_counts()

USA                    104
New York                71
United States           50
London                  45
Canada                  29
                      ... 
MontrÌ©al, QuÌ©bec       1
Montreal                 1
ÌÏT: 6.4682,3.18287      1
Live4Heed??              1
Lincoln                  1
Name: location, Length: 3341, dtype: int64

In [8]:
df['location'] = df['location'].fillna('Unknown')

In [9]:
df['merged'] = df['location'] + " " + df['text']

In [10]:
df['merged'].sample(4)

1097    IDN London Life: photos of a beautiful bombed-...
1352    Los Angeles Ali you flew planes and ran into b...
6039    Mogadishu, Somalia Exploration Takes Seismic S...
1485    Azeroth Chances are many of us are still diggi...
Name: merged, dtype: object

##NLP

In [11]:
def clearing(text):
    
    text=text.lower()                    # Buradan sonraki 4 satırd ise NLP methodlarını uygulayabilmek adına
                                         # bütün veriyi küçük harflere çevirdik ve içlerinden numerik 
                                         # verileri ve de sembolleri attık
    text=text.replace("[^\w\s]","") 
    text=text.replace("\d+","") 
    text=text.replace("\n"," ").replace("\r","") 
    text=text.replace("\&\#[0-9]+\;",'')
    
    return text

In [12]:
df['merged'] = df['merged'].apply(clearing)

In [13]:
df['merged'].replace({'r"\&\#[0-9]+\;"':''},regex=True,inplace=True)

In [14]:
df['merged'].replace('r"\&\#[0-9]+\;')

0       unknown our deeds are the reason of this #eart...
1          unknown forest fire near la ronge sask. canada
2       unknown all residents asked to 'shelter in pla...
3       unknown 13,000 people receive #wildfires evacu...
4       unknown just got sent this photo from ruby #al...
                              ...                        
7608    unknown two giant cranes holding a bridge coll...
7609    unknown @aria_ahrary @thetawniest the out of c...
7610    unknown m1.94 [01:04 utc]?5km s of volcano haw...
7611    unknown police investigating after an e-bike c...
7612    unknown the latest: more homes razed by northe...
Name: merged, Length: 7613, dtype: object

In [15]:
df['merged'].replace({'r"\&\#[0-9]+\;"':''},regex=True,inplace=True)

In [16]:
df['merged']

0       unknown our deeds are the reason of this #eart...
1          unknown forest fire near la ronge sask. canada
2       unknown all residents asked to 'shelter in pla...
3       unknown 13,000 people receive #wildfires evacu...
4       unknown just got sent this photo from ruby #al...
                              ...                        
7608    unknown two giant cranes holding a bridge coll...
7609    unknown @aria_ahrary @thetawniest the out of c...
7610    unknown m1.94 [01:04 utc]?5km s of volcano haw...
7611    unknown police investigating after an e-bike c...
7612    unknown the latest: more homes razed by northe...
Name: merged, Length: 7613, dtype: object

##Modelling

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from textblob import TextBlob
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer('english') 

def split_into_lemmas(text):
    
    text = str(text).lower() 
    
    words = TextBlob(text).words
    
    return [stemmer.stem(word) for word in words]

In [18]:
x,y=df['merged'],df['target']

In [19]:
y

0       1
1       1
2       1
3       1
4       1
       ..
7608    1
7609    1
7610    1
7611    1
7612    1
Name: target, Length: 7613, dtype: int64

In [20]:
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=50)

##Vectorizing

In [21]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [22]:
vect=CountVectorizer(lowercase=True, stop_words='english', ngram_range=(1,2), analyzer=split_into_lemmas)
x_train_dtm=vect.fit_transform(x_train)
x_test_dtm=vect.transform(x_test)

In [23]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

In [24]:
b=MultinomialNB()
model=b.fit(x_train_dtm,y_train)
b_predict=b.predict(x_test_dtm)

In [25]:
accuracy_score(y_test,b_predict)

0.7935924369747899